# Sachsen-Anhalt

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Sachsen-Anhalt is `DEE`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [ ]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from glob import glob
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict, Tuple
from datetime import datetime as dt
from dateparser import parse
import warnings
from io import BytesIO

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [ ]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Sachsen-Anhalt').input_path
BASE

## Parse data

We do not have a Metadata file, but one Excel file for each station. Thus we need to parse each metadata individually and collect.

In [ ]:
files = glob(os.path.join(BASE, 'LHW_*.DGJ'))
print(f"Found {len(files)} files.")

Check the Header on the first file, as we don't have any other Metadata file. We need a ZRXP parser. Couldn't find something simple quickly, so I write my own. https://prozessing.tbbm.at/zrxp/zrxp3.0_en.pdf this is the ZRXP specification. I will only implement the relevant metadata

In [ ]:
# get the names from the header
# I will skip the energy market headers and remote logger headers
HEADER = dict(
    SANR='Alphanumerical station number',
    SNAME='Station name',
    SWATER='River name',
    CMW='Values per day for equidistant time series values',
    CNAME='Parameter name',
    CNR='Parameter number',
    CUNIT='Unit of the data value column',
    RINVAL='Value for missing or invalid data record',
    RTIMELVL='Time series time level',
    TZ='time zone of all time stamps in the time series block, both header and data',
    LAYOUT='specifies the column layout for the ZRXP data'
)

def extract_file(path: str) -> Tuple[Dict[str, str], pd.DataFrame]:
    # Get the header lines first
    collection = []
    headerlines = 0

    # first read the head
    with open(path, 'rb') as f:
        # go for each line
        for l in f.readlines():
            if not l.decode('latin1').startswith('#'):
                break
            else:
                # collect the header
                collection.extend([_ for _ in l.decode('latin1').replace('#', '').split('|*|') if _ not in ('', '\n', '\r\n')])
                headerlines += 1

    # create metadata container
    meta = {}

    # go for each collected header
    for co in collection:
        HEAD = [k for k in HEADER.keys() if co.startswith(k)]
        if len(HEAD) == 1:
            HEAD = HEAD[0]
            meta[HEAD] = co.replace(HEAD, '')
        elif len(HEAD) == 0:
            if 'COMMENT' in meta:
                meta['COMMENT'] += f" {co}"
            else:
                meta["COMMENT"] = co
        elif len(HEAD) > 1:
            warnings.warn(f"Can't parse header {co}")

    # now the data
    header = meta['LAYOUT'].strip('()').split(',') if 'LAYOUT' in meta else [0, 1, 2, 3, 4, 5]
    df = pd.read_csv(path, encoding='latin1', sep=' ', header=None, skiprows=headerlines, names=header, parse_dates=[0], na_values=int(meta.get('RINVAL', '-777')))

    return meta, df

extract_file(files[-1])


Go go for all files

In [ ]:
# container
meta = []
raw_data = []

with warnings.catch_warnings(record=True) as warn:
    for fname in tqdm(files):
        m, df = extract_file(fname)
        meta.append(m)
        raw_data.append(df)

print(f"Parsed {len(meta)} files with {len(warn)} warnings.")

# create metadata

This should be pretty straightforward, but maybe not super-helpful

In [ ]:
metadata = pd.DataFrame(meta)
metadata

In [ ]:
id_column = 'SANR'

Get all status

In [ ]:
data = []
status = []
int_type = []

for m, df in zip(meta, raw_data):
    # get status
    for s in df.status.unique():
        if s not in status:
            status.append(s)
    
    # get interpolation types
    for t in df.interpolation_type.unique():
        if t not in int_type:
            int_type.append(t)
    
    # make the df
    out = df.iloc[:, :2].copy()
    out.columns = ['date', 'q' if m['CNAME'].startswith('Q') else 'w']
    out['flag'] = np.NaN
    data.append(out)

print(f"Stauts:              {status}")
print(f"Interpolation types: {int_type}")

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [ ]:
with Bundesland('Sachsen-Anhalt') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    # go for each    
    for m, df in tqdm(zip(meta, data), total=len(meta)):
        
        # get the provider id
        provider_id = str(m[id_column])
        bl.save_timeseries(df, provider_id)

    # check if there were warnings (there are warnings)
    if len(warn) > 0:
        log_path = bl.save_warnings(warn)
        print(f"There were warnings during the processing. The log can be found at: {log_path}")
